# Project: NIP
# Subroject: Sylhet Floods, June 2022
## Script: summarize_accuracyVsLabel.ipynb
This script summarizes the accruacy of the flood mapping algorithms and FFWC map against the hand labels.

In [1]:
from pathlib import Path
import os
import sys
import pandas as pd
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from rasterio.plot import show
from rasterio.mask import mask
from itertools import chain
from pyproj import Transformer
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors
from datetime import datetime
import geopandas as gpd
import copy
import collections
from shapely.geometry import mapping
import pycrs
from shapely.geometry import Polygon
from functools import reduce
import shutil

In [2]:
# Set the root path
rootPath = Path('Z:/media/mule/Projects/NASA/NIP/Data')

In [3]:
# Add path for the Helpers modules
module_path = os.path.abspath(os.path.join('C:/Users/alexsaunders/Documents/01_uoa/04_git/NIP/Sylhet/'))
if module_path not in sys.path:
    sys.path.append(module_path)

## PART 1: Get the already created evaluation metrics

In [42]:
# Read in the result of the accuracy evaluation
accuracyMaps = pd.read_csv(rootPath/'Table/SylhetConfusionMatrix/confusMatrixVsLabels_20220524-20220715_v2.csv')
accuracyFFWC = pd.read_csv(rootPath/'Table/SylhetConfusionMatrix/confusMatrixVsLabels_FFWC.csv')

In [43]:
accuracyMaps['label']=accuracyMaps.labelDate.map(str)+'_'+accuracyMaps.labelTile.map(str)
accuracyFFWC['label']=accuracyFFWC.labelDate.map(str)+'_'+accuracyFFWC.labelTile.map(str)

In [44]:
# Decide on a single best threshold for FFWC map
FFWCthresh=0.0001
accuracyFFWC=accuracyFFWC[accuracyFFWC.DepthThreshold==FFWCthresh]
accuracyFFWC=accuracyFFWC.drop(columns=['DepthThreshold'])

## PART 2: Aggregate by MapSource (i.e. all dates)

In [49]:
metrics=['Precision','Recall','Accuracy','IOU']

In [50]:
scoresMaps = accuracyMaps.groupby(by='MapSource')[metrics].mean()
scoresFFWC = accuracyFFWC.groupby(by='MapSource')[metrics].mean()
scores=pd.concat(objs=[scoresMaps,scoresFFWC])

In [56]:
scores.sort_values('IOU', ascending=False)

,Precision,Recall,Accuracy,IOU
MapSource,,,,
Devries,0.848924,0.753231,0.819787,0.677602
FFWC,0.671212,0.967839,0.687735,0.664379
NASAIMPACT,0.934219,0.695506,0.822906,0.662287
GFM,0.927819,0.523928,0.732585,0.512674


## PART 3: Aggregate by MapSource and label date

In [52]:
scoresMaps = accuracyMaps.groupby(by=['labelDate','MapSource'])[metrics].mean()
scoresFFWC = accuracyFFWC.groupby(by=['labelDate','MapSource'])[metrics].mean()
scoresByDate=pd.concat(objs=[scoresMaps,scoresFFWC])

In [58]:
scoresByDate#.sort_values('IOU', ascending=False)

Precision    Recall  Accuracy       IOU
labelDate MapSource                                          
20220524  Devries      0.959126  0.831911  0.847495  0.805364
          GFM          0.995420  0.621808  0.707957  0.619797
          NASAIMPACT   0.984254  0.809977  0.845835  0.799312
20220530  Devries      0.852633  0.734672  0.805756  0.647657
          GFM          0.957818  0.539214  0.761929  0.528152
          NASAIMPACT   0.911171  0.710189  0.822213  0.662171
20220714  Devries      0.744815  0.696574  0.806082  0.585440
          GFM          0.840854  0.421466  0.730709  0.401579
          NASAIMPACT   0.905137  0.578338  0.802466  0.536776
20220524  FFWC         0.783945  0.990405  0.778922  0.778333
20220530  FFWC         0.632961  0.964783  0.622361  0.617050
20220714  FFWC         0.599749  0.949702  0.658626  0.599363